# Time Series Data Cleaning  
**Author:** Chao Lu  
**Last update:** 2024/07/01

# Introduction

This notebook addresses the challenge of cleaning a time series dataset, which contains irregular entries. The dataset should ideally have data recorded at consistent 5-minute intervals (e.g., 11:45, 11:50, 11:55, 12:00, etc.). However, it has been observed that there are extraneous entries at times like 11:56 or randomly at xx:02, which disrupt the expected 5-minute sequence. This inconsistency in time intervals can lead to issues in subsequent data analysis and forecasting tasks.

**Goals**
1. **Identify Extraneous Entries**: Detect and identify data points that do not adhere to the standard 5-minute interval.
2. **Maintain 5-minute Spacing**: Ensure that the dataset only includes entries at 5-minute intervals, removing all extraneous entries.
3. **Ensure Data Completeness**: After removing the irregular entries, check for any gaps in the 5-minute intervals and address any missing data points to maintain a complete and consistent dataset for accurate analysis.

By accomplishing these goals, the dataset will be structured for better usability in time series analysis, ensuring reliability and consistency in data-driven decisions.

## Load the original dataset

The first step is to install and import the necessary packages for this task. Define the file path and file name to load the dataset effectively.

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun May 12 17:15:28 2024
@author: chao lu
"""

# Import necessary libraries
import pandas as pd
import os

# Define the directory path and filename of the dataset
path = r'C:\Users\chaolu\workspace\time series data'
filename = 'weather_data_utf8.csv'
target_file = os.path.join(path, filename)

# Load the dataset
df = pd.read_csv(target_file)

## Identify and Handle Inconsistent Timestamps

This section focuses on identifying and correcting inconsistencies in timestamp intervals within the dataset. The goal is to ensure that all timestamps align precisely with expected 5-minute intervals.

**Convert Date/Time to DateTime:** First, the 'Date/Time' column is converted into a datetime format, manually adding the year 2023 for consistency and to facilitate datetime operations.

In [2]:
df['Date/Time'] = pd.to_datetime(df['Date/Time'].apply(lambda x: x + ', 2023'), format='%b %d, %I:%M %p, %Y')

**Round Timestamps to Nearest 5 Minutes:** Each timestamp is rounded to the nearest 5-minute interval to standardize the data entries. This rounded time is then used to identify timestamps that precisely match the expected 5-minute marks.

In [3]:
df['Rounded Time'] = df['Date/Time'].dt.round('5min')
df_on_interval = df[df['Rounded Time'] == df['Date/Time']]

**Determine Range of Timestamps:** By finding the earliest and latest timestamps within these rounded times, we establish the range of our analysis.

In [4]:
start_time = df_on_interval['Date/Time'].min()
end_time = df_on_interval['Date/Time'].max()

**Generate Expected Sequence of Timestamps:** A complete sequence of 5-minute intervals is generated between the earliest and latest times identified. This sequence represents the ideal timestamps expected in the dataset.

In [5]:
expected_times = pd.date_range(start=start_time, end=end_time, freq='5min')

**Filter Out Inconsistent Entries:** The dataset is then filtered to include only those entries whose timestamps match exactly with the generated 5-minute intervals. This step removes any rows that do not conform to the expected timing, such as random entries at non-standard times.

In [6]:
df_clean = df[df['Date/Time'].isin(expected_times)]

**Report and Remove Inconsistencies:** Calculate the number of rows that had inconsistent timestamps and were removed. This measure gives insight into the extent of the cleaning required and confirms the application of the specified data handling method.

In [7]:
num_inconsistent = len(df) - len(df_clean)
print(f"Total number of inconsistent format rows: {num_inconsistent}")
print("Dealing method: Entire row remove")

Total number of inconsistent format rows: 2460
Dealing method: Entire row remove


## Remove Duplicates

This section is dedicated to identifying and removing duplicate entries in the dataset to ensure each timestamp is represented uniquely. Maintaining a singular record for each 5-minute interval is crucial for the accuracy of time series analysis.

**Sort Data by Date/Time:** First, the data is sorted in ascending order by 'Date/Time'. This step ensures that when duplicates are identified, the selection of which entry to retain (the latest occurrence) is accurately applied.

In [8]:
df_clean = df_clean.sort_values(by='Date/Time', ascending=True)

**Identify Duplicates:** Before removing duplicates, it's important to first identify them. This is done using the duplicated method, which flags all entries that have a duplicate 'Date/Time' stamp, allowing for an accurate count of duplicates before any are removed.

In [9]:
duplicates_before = df_clean.duplicated(subset=['Date/Time'], keep=False)
num_duplicates = duplicates_before.sum()

**Remove Duplicate Entries:** Duplicates are removed, keeping only the last occurrence of each timestamp. This approach is chosen because the latest record is often the most accurate or complete in many data collection processes, ensuring that the best possible data is retained.

In [10]:
df_clean = df_clean.drop_duplicates(subset=['Date/Time'], keep='last')

**Report Removal of Duplicates:** After the duplicates are removed, the number of duplicates that were found and handled is reported. This transparency helps in understanding the impact of this step on the dataset's integrity and size.

In [11]:
print(f"Total number of duplicate rows: {num_duplicates}")
print("Dealing method: Retain the latest record for duplicates")

Total number of duplicate rows: 24
Dealing method: Retain the latest record for duplicates


## Data Overview

In this section, we will check the data types and may change them based on the nature of the data. For example, the 'Wind_Speed_kph' should not be of type 'object'. We will inspect the data and convert it to float numbers.

In [12]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26313 entries, 0 to 28783
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date/Time               26313 non-null  datetime64[ns]
 1   Temp_C                  26280 non-null  float64       
 2   Dew_Point_C             26280 non-null  float64       
 3   RH                      26280 non-null  float64       
 4   Heat_Index_C            99 non-null     float64       
 5   Wind_Chill_C            6186 non-null   float64       
 6   Wind_Dir                26205 non-null  object        
 7   Wind_Speed_kph          26313 non-null  object        
 8   Visibility_km           26278 non-null  float64       
 9   Weather                 1445 non-null   object        
 10  Clouds_x100_ft          26304 non-null  object        
 11  Sea_Level_Pressure _mB  0 non-null      float64       
 12  Station_Pressure_mB     26313 non-null  float64    

To convert the `df_clean` 'Wind_Speed_kph' column to float numbers, we need to handle the entries that contain the 'G' notation, which likely represents gusts of wind. We will remove any characters after 'G' and retain only the numerical part before 'G'. After cleaning the data to ensure it contains only numerical values, we will convert the entire column to float.

In [13]:
# Remove the gust notation and convert to float
df_clean['Wind_Speed_kph'] = df_clean['Wind_Speed_kph'].str.extract(r'(\d+)').astype(float)

## Handle Missing Timestamps

In this section, we ensure that the dataset has a complete series of records for every expected 5-minute interval by identifying and addressing any missing timestamps. This step is crucial to prevent any gaps that could affect time series analysis and forecasting accuracy.

**Generate a Complete Time Range DataFrame:** First, a DataFrame is created that lists all expected timestamps based on the previously established 5-minute interval range. This DataFrame serves as a template to ensure no gaps exist in the final dataset.

In [14]:
all_times = pd.DataFrame(expected_times, columns=['Date/Time'])

**Merge with Cleaned Data:** The DataFrame containing all expected timestamps is merged with the cleaned data. This merge is performed as a left join, which ensures that all timestamps in all_times are included in the resulting DataFrame, df_final, with data from df_clean aligned where available.

In [15]:
df_final = pd.merge(all_times, df_clean, on='Date/Time', how='left')

**Identify Missing Timestamps:** By comparing the complete time range with the timestamps present in the cleaned data, we can identify any missing timestamps. This step is essential for quantifying the extent of missing data.

In [16]:
missing_timestamps = all_times[~all_times['Date/Time'].isin(df_clean['Date/Time'])]
num_missing_timestamps = len(missing_timestamps)
print(f"Number of missing timestamps: {num_missing_timestamps}")

Number of missing timestamps: 183


**Apply Filling Strategies:** For the columns identified as critical for analysis (e.g., temperature, dew point, and relative humidity), missing values are interpolated linearly from the nearest existing data points. This method of interpolation provides a reasonable approximation for missing data, ensuring continuity in the dataset's environmental conditions.

In [17]:
columns_to_fill = ['Temp_C', 'Dew_Point_C', 'RH', 'Wind_Speed_kph', 'Station_Pressure_mB', 'Altimeter_Setting_mB']  # Example columns
for column in df_final.columns:
    if column in columns_to_fill:
        df_final[column] = df_final[column].interpolate(method='linear', limit_direction='both', limit_area='inside')

**Leave Non-Essential Columns with NaN:** Columns not specified for interpolation are left with NaN where data is missing. This decision is made to avoid introducing potentially misleading data into the analysis.

**Report on Data Handling Method:** Finally, the chosen method for handling missing data and the columns affected are clearly documented. This transparency helps in understanding how data integrity is maintained and ensures reproducibility of the dataset preparation process.

In [18]:
print("Dealing method: Apply linear interpolation to fill missing values in specified columns: " + ', '.join(columns_to_fill))

Dealing method: Apply linear interpolation to fill missing values in specified columns: Temp_C, Dew_Point_C, RH, Wind_Speed_kph, Station_Pressure_mB, Altimeter_Setting_mB


## Save the Final Cleaned Data

This section outlines the final steps to save the thoroughly cleaned and processed dataset, ensuring that the data is ready for further analysis or sharing. This is a critical step in preserving the integrity of the cleaned dataset and ensuring its usability in future applications.

**Remove Unnecessary Columns:** First, any columns added during the data cleaning process that are no longer needed for analysis are removed. For this dataset, the 'Rounded Time' column, which was used to align and check the timestamps, is removed to streamline the data structure.

In [19]:
df_final = df_final.drop(columns=['Rounded Time'])

**Specify the Output File Path:** Define the path where the cleaned dataset will be saved. This path includes the directory and the filename. The filename is chosen to reflect that the dataset is the final version after all cleaning steps have been applied.

In [20]:
output_file_path = os.path.join(path, 'df_final.csv')

**Save the Dataset to a CSV File:** The cleaned dataset is saved to a CSV file. The index=False parameter is used to prevent pandas from writing row numbers (index) into the CSV file, which keeps the file clean of any unnecessary data. The encoding 'ISO-8859-1' is specified to ensure compatibility with various systems and software that might read this CSV file.

In [21]:
df_final.to_csv(output_file_path, index=False, encoding='ISO-8859-1')

**Confirm Successful Save:** Finally, a confirmation message is printed to indicate that the data has been successfully saved to the specified location. This step provides immediate feedback that the save operation has been completed without issues.

In [22]:
print("Cleaned time series data saved to:", output_file_path)

Cleaned time series data saved to: C:\Users\chaolu\workspace\time series data\df_final.csv
